In [1]:
import ee
import geemap

try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# Create the map, ROI is assumed to be already defined
Map = geemap.Map(center=[0, 0], zoom=2)
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
if Map.user_rois is None:
    print("No user ROI found. Please draw an ROI on the map before running this cell.")
else:
    # Check the number of features in the collection
    roi_size = Map.user_rois.size().getInfo()
    if roi_size == 0:
        print("No user ROI found. Please draw an ROI on the map before running this cell.")
    else:
        # We'll take the first feature from the FeatureCollection and get its geometry
        first_feature = Map.user_rois.first()
        roi = first_feature.geometry()
        print("ROI successfully retrieved!")
        print("Feature info:", first_feature.getInfo())
        

ROI successfully retrieved!
Feature info: {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-119.245605, 34.732584], [-119.185181, 34.011689], [-117.405396, 32.212801], [-116.065063, 33.14675], [-117.366943, 34.443159], [-118.295288, 34.809293], [-119.245605, 34.732584]]]}, 'id': '0', 'properties': {}}


In [5]:
import datetime

start_date = '2024-09-01'
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

s2_collection = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(roi)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
)

s2_median = s2_collection.median()
ndvi = s2_median.normalizedDifference(["B8", "B4"]).rename("NDVI")

forest_mask = ndvi.gt(0.6)
shrub_mask = ndvi.gt(0.3).And(ndvi.lte(0.6))
classification = (forest_mask.multiply(1).add(shrub_mask.multiply(2))).rename("landcover")

fire_potential = ee.Image(1).subtract(ndvi).rename("fire_potential")

analysis_image = (
    s2_median.select(["B4", "B3", "B2"])
    .addBands([classification, fire_potential, ndvi])
    .clip(roi)
)

rgb_vis = {
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 3000,
}

class_vis = {
    "min": 0,
    "max": 2,
    "palette": ["ffffff", "006400", "7FFF00"]  # white=none, darkgreen=forest, greenyellow=shrub
}

fire_vis = {
    "min": 0,
    "max": 1,
    "palette": ["blue", "yellow", "red"]
}

# These are 'visual' RGB images for each layer
s2_composite_vis = s2_median.visualize(**rgb_vis)
landcover_vis_img = analysis_image.select("landcover").visualize(**class_vis)
fire_potential_vis_img = analysis_image.select("fire_potential").visualize(**fire_vis)


In [8]:
def blend_images(base_rgb, overlay_rgb, alpha):
    """
    base_rgb, overlay_rgb: ee.Image of 3 visualization bands (R,G,B).
    alpha: float in [0,1], how much of overlay is visible on top of base.
    Returns a 3-band ee.Image that is the alpha blend of the two.
    """
    alpha_img = ee.Image.constant(alpha)
    inv_alpha = ee.Image.constant(1.0).subtract(alpha_img)
    # Blend per band: result = base*(1-alpha) + overlay*(alpha)
    return base_rgb.multiply(inv_alpha).add(overlay_rgb.multiply(alpha_img))


def annotate(img, text):
    """
    Annotate an ee.Image with text in the top-left corner using geemap.
    The default style is white text with black outline.
    """
    return geemap.annotate_image(
        image=img,
        text=text,
        font_size=20,
        font_color="white",
        outline_color="black",
        outline_width=2,
        position="left"  # top-left
    )


In [9]:
images_list = []

# Phase 1: forest fade-in from 0..1 in steps of 0.1
for i in range(11):
    forest_alpha = i / 10.0
    fire_alpha   = 0.0        # Fire off
    blended = blend_images(s2_composite_vis, landcover_vis_img, forest_alpha)
    label_text = f"Forest/Shrubland Opacity: {int(forest_alpha*100)}%"
    # Annotate
    annotated = annotate(blended, label_text)
    images_list.append(annotated)

# Phase 2: now forest remains at 100%, fade-in fire from 0..1
for i in range(11):
    fire_alpha   = i / 10.0
    # Start with S2 + forest fully on
    base_forest = blend_images(s2_composite_vis, landcover_vis_img, 1.0)
    # Then blend the fire overlay
    blended = blend_images(base_forest, fire_potential_vis_img, fire_alpha)
    label_text = (
        f"Forest/Shrubland: 100%  |  Fire Potential Opacity: {int(fire_alpha*100)}%"
    )
    annotated = annotate(blended, label_text)
    images_list.append(annotated)

# Convert to an ImageCollection
animation_col = ee.ImageCollection.fromImages(images_list)


AttributeError: module 'geemap' has no attribute 'annotate_image'

In [16]:
s2_median = s2_collection.median()
ndvi = s2_median.normalizedDifference(["B8", "B4"]).rename("NDVI")

forest_mask = ndvi.gt(0.6)
shrub_mask = ndvi.gt(0.3).And(ndvi.lte(0.6))
classification = (forest_mask.multiply(1).add(shrub_mask.multiply(2))).rename("landcover")

fire_potential = ee.Image(1).subtract(ndvi).rename("fire_potential")

analysis_image = (
    s2_median.select(["B4", "B3", "B2"])
    .addBands([classification, fire_potential, ndvi])
    .clip(roi)
)

# Visualization dictionaries
rgb_vis = {
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 3000,
}
class_vis = {
    "min": 0,
    "max": 2,
    "palette": ["ffffff", "006400", "7FFF00"] 
}
fire_vis = {
    "min": 0,
    "max": 1,
    "palette": ["blue", "yellow", "red"]
}

# Create 3-band "visual" images for S2, Forest, Fire
s2_composite_vis = s2_median.visualize(**rgb_vis)
landcover_vis_img = analysis_image.select("landcover").visualize(**class_vis)
fire_potential_vis_img = analysis_image.select("fire_potential").visualize(**fire_vis)

def blend_images(base_rgb, overlay_rgb, alpha):
    """Alpha-blend two ee.Image(3-band) using alpha in [0..1]."""
    alpha_img = ee.Image.constant(alpha)
    inv_alpha = ee.Image.constant(1.0).subtract(alpha_img)
    return base_rgb.multiply(inv_alpha).add(overlay_rgb.multiply(alpha_img))

# Build frames by blending, but skip text annotation
images_list = []

# Phase 1: forest fade-in from 0..1 in steps of 0.1
for i in range(11):
    forest_alpha = i / 10.0
    blended = blend_images(s2_composite_vis, landcover_vis_img, forest_alpha)
    images_list.append(blended)

# Phase 2: forest at 100%, fade-in fire from 0..1
for i in range(11):
    fire_alpha = i / 10.0
    base_forest = blend_images(s2_composite_vis, landcover_vis_img, 1.0)
    blended = blend_images(base_forest, fire_potential_vis_img, fire_alpha)
    images_list.append(blended)

animation_col = ee.ImageCollection.fromImages(images_list)

# Export the GIF without text
out_gif = "forest_fire_no_text.gif"
video_args = {
    "dimensions": 600,
    "region": roi,
    "framesPerSecond": 2,
    "crs": "EPSG:3857",
}

for i, img in enumerate(images_list):
    task = ee.batch.Export.image.toDrive(
        image=img,
        description=f"Frame_{i:02d}",
        folder="GEE_Frames",  # Specify your desired Google Drive folder
        fileNamePrefix=f"frame_{i:02d}",
        region=roi,
        scale=1000,
        crs="EPSG:4326"
    )
    task.start()
    print(f"Exporting frame {i+1}/{len(images_list)}...")


Exporting frame 1/22...
Exporting frame 2/22...
Exporting frame 3/22...
Exporting frame 4/22...
Exporting frame 5/22...
Exporting frame 6/22...
Exporting frame 7/22...
Exporting frame 8/22...
Exporting frame 9/22...
Exporting frame 10/22...
Exporting frame 11/22...
Exporting frame 12/22...
Exporting frame 13/22...
Exporting frame 14/22...
Exporting frame 15/22...
Exporting frame 16/22...
Exporting frame 17/22...
Exporting frame 18/22...
Exporting frame 19/22...
Exporting frame 20/22...
Exporting frame 21/22...
Exporting frame 22/22...


In [31]:
frame_directory = r"frames" 

In [34]:
frame_files = sorted(glob.glob(rf"{frame_directory}\frame_*.tif"))

In [35]:
frame_files

['frames\\frame_00.tif',
 'frames\\frame_01.tif',
 'frames\\frame_02.tif',
 'frames\\frame_03.tif',
 'frames\\frame_04.tif',
 'frames\\frame_05.tif',
 'frames\\frame_06.tif',
 'frames\\frame_07.tif',
 'frames\\frame_08.tif',
 'frames\\frame_09.tif',
 'frames\\frame_10.tif',
 'frames\\frame_11.tif',
 'frames\\frame_12.tif',
 'frames\\frame_13.tif',
 'frames\\frame_14.tif',
 'frames\\frame_15.tif',
 'frames\\frame_16.tif',
 'frames\\frame_17.tif',
 'frames\\frame_18.tif',
 'frames\\frame_19.tif',
 'frames\\frame_20.tif',
 'frames\\frame_21.tif']

In [42]:
valid_frames = []

for file in frame_files:
    try:
        img = Image.open(file)
        valid_frames.append(img)
    except Exception as e:
        print(f"Skipping {file}: {e}")

if valid_frames:
    valid_frames[0].save(
        "forest_fire_animation.gif",
        save_all=True,
        append_images=valid_frames[1:],
        duration=2000,
        loop=0
    )
    print("GIF created successfully: forest_fire_animation.gif")
else:
    print("No valid frames to create GIF.")


Skipping frames\frame_01.tif: cannot identify image file 'frames\\frame_01.tif'
Skipping frames\frame_02.tif: cannot identify image file 'frames\\frame_02.tif'
Skipping frames\frame_03.tif: cannot identify image file 'frames\\frame_03.tif'
Skipping frames\frame_04.tif: cannot identify image file 'frames\\frame_04.tif'
Skipping frames\frame_05.tif: cannot identify image file 'frames\\frame_05.tif'
Skipping frames\frame_06.tif: cannot identify image file 'frames\\frame_06.tif'
Skipping frames\frame_07.tif: cannot identify image file 'frames\\frame_07.tif'
Skipping frames\frame_08.tif: cannot identify image file 'frames\\frame_08.tif'
Skipping frames\frame_09.tif: cannot identify image file 'frames\\frame_09.tif'
Skipping frames\frame_11.tif: [Errno 2] No such file or directory: 'frames\\frame_11.tif'
Skipping frames\frame_12.tif: cannot identify image file 'frames\\frame_12.tif'
Skipping frames\frame_13.tif: cannot identify image file 'frames\\frame_13.tif'
Skipping frames\frame_14.tif: 

In [36]:
frames = [Image.open(f) for f in frame_files]

# Save as GIF
output_gif = "forest_fire_animation.gif"  # Name of the output GIF
frames[0].save(
    output_gif,
    save_all=True,
    append_images=frames[1:],  # Add remaining frames
    duration=500,  # 500ms per frame (adjust for animation speed)
    loop=0         # 0 = infinite loop
)

print(f"GIF created: {output_gif}")


UnidentifiedImageError: cannot identify image file 'frames\\frame_01.tif'

In [38]:
for file in frame_files:
    try:
        img = Image.open(file)
        print(f"{file}: mode={img.mode}, size={img.size}")
    except Exception as e:
        print(f"Error opening {file}: {e}")


frames\frame_00.tif: mode=RGB, size=(355, 290)
Error opening frames\frame_01.tif: cannot identify image file 'frames\\frame_01.tif'
Error opening frames\frame_02.tif: cannot identify image file 'frames\\frame_02.tif'
Error opening frames\frame_03.tif: cannot identify image file 'frames\\frame_03.tif'
Error opening frames\frame_04.tif: cannot identify image file 'frames\\frame_04.tif'
Error opening frames\frame_05.tif: cannot identify image file 'frames\\frame_05.tif'
Error opening frames\frame_06.tif: cannot identify image file 'frames\\frame_06.tif'
Error opening frames\frame_07.tif: cannot identify image file 'frames\\frame_07.tif'
Error opening frames\frame_08.tif: cannot identify image file 'frames\\frame_08.tif'
Error opening frames\frame_09.tif: cannot identify image file 'frames\\frame_09.tif'
frames\frame_10.tif: mode=RGB, size=(355, 290)
frames\frame_11.tif: mode=RGB, size=(355, 290)
Error opening frames\frame_12.tif: cannot identify image file 'frames\\frame_12.tif'
Error ope